#### 전략 1 [3일 모멘텀 단기 전략]  
마켓타이밍 ==True  
거래대금 100억이상  
3일째 가격상승이 되는 종목 매수후 3일뒤 매도  

3일간 상승한 모멘텀이 앞으로 3일간 지속될 것 이라는 가정  

In [6]:
import pandas as pd 
df_raw = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/mom.csv', index_col=0)

In [7]:
df_raw = df_raw.drop(df_raw.iloc[:,8:],axis=1)

In [12]:
# make momentum_signal
def make_momentum_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        con1 = (df.loc[idx[0]:idx[-1],'종가'].shift(-3))
        con2 = (df.loc[idx[0]:idx[-1],'종가'].shift(-2))
        con3 = (df.loc[idx[0]:idx[-1],'종가'].shift(-1))
        df.loc[idx[0]:idx[-1], 'momentum'] = (con1<con2)&(con2<con3)&(con3<(df.loc[idx[0]:idx[-1], '종가']))
    return df
        

In [10]:
df = make_momentum_signal(df_raw)

In [13]:
# market_timming 병합작업
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [23]:
start = df[df['종목명']=='삼성전자'].head(1)['날짜'].values[0]
end = df[df['종목명']=='삼성전자'].tail(1)['날짜'].values[0]

In [25]:
df_mt.reset_index(drop=True, inplace=True)

In [22]:
df_mt

In [24]:
df_mt = making_mt(start,end,'2001')
df_mt['날짜']=df_mt.index

In [34]:
df_mt['날짜'] = df_mt['날짜'].astype('object')

In [41]:
df_join = pd.merge(df,df_mt,how='left', on='날짜')

#### 거래대금 필터만우선적용..

In [49]:
df['vol_condition'] = df['누적거래대금'] >=10000000000

#### 최종시그널

In [52]:
def make_buysell(df):
    df['buy'] = (df['vol_condition'])&(df['momentum'])
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'sell'] = df.loc[idx[0]:idx[-1],'buy'].shift(3)
    return df

In [53]:
df = make_buysell(df)

In [55]:
code_kospi = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kosdaq_codes.csv')

In [56]:
code = pd.concat([code_kospi,code_kosdaq])
code.rename(columns={'회사명':'종목명'},inplace=True)
code['종목코드'] = code['종목코드'].astype('str')

In [58]:
df = df.merge(code,on='종목명',how='inner')
df.rename(columns={'종목코드':'코드'},inplace=True)
df['구매수량']=3

#### 백테스트

In [60]:
df['sell'].unique()

array([nan, True, False], dtype=object)

In [61]:
import sys
sys.path.append('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test')
from BackTesting_System.backtesting import BackTesting, Utils

In [62]:
bt1 = BackTesting()
ut = Utils()

In [63]:
bt1.MONEY = 10000000

In [64]:
bt1.buy_sell(df=df,buy='buy',sell='sell')

In [65]:
bt1.update_information()

In [66]:
bt1.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2018-02-01,047810,51700,3,218.0,매수
1,2018-02-01,093370,10600,3,45.0,매수
2,2018-02-01,034220,31650,3,133.0,매수
3,2018-02-01,071050,87000,3,367.0,매수
4,2018-02-01,051910,418500,3,1764.0,매수
5,2018-02-01,000880,47000,3,198.0,매수
6,2018-02-01,001060,43290,3,183.0,매수
7,2018-02-01,001040,173034,3,729.0,매수
8,2018-02-01,003520,9610,3,41.0,매수
9,2018-02-01,950160,109604,3,462.0,매수


In [67]:
bt1.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-01,3128943,0.0,4397.0,0.0
1,2018-02-02,3932307,421005,6116.0,-18786
2,2018-02-05,1784508,600528,3352.0,-38649
3,2018-02-07,1208604,0.0,1699.0,0.0
4,2018-02-08,951273,0.0,1338.0,0.0


In [68]:
bt1.ECONO_INFORM['실현손익'].sum()

-57435.0

In [69]:
bt1.ECONO_INFORM['실현손익'].sum()/2000000000*100

-0.00287175

#### 문제 : 매수종목이 너무많음 5일만에 투자가 끝남  
해결책 : 매수종목을 더 추려야함 + 기술적지표 추가  

#### 매도조건 바꾸기 : 현재 3일뒤 매도, 